# Team Introduction
Our group is comprised of Braden Anderson, Hien Lam, and Tavin Weeda.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import pickle 

from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, multilabel_confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.ensemble import RandomForestClassifier

# Data Preparation Part 1
- Define and prepare your class variables. 
- Use proper variable representations (int, float, one-hot, etc.). 
- Use pre-processing methods (as needed) for dimensionality reduction, scaling, etc. 
- Remove variables that are not needed/useful for the analysis.
(10)

## Read, clean the data

In [2]:
# Read in data from github
url_accident = "https://github.com/BradenAnderson/Accident_Severity_Prediction/blob/main/Data/accident.csv.gz?raw=tr"
url_vehicle = "https://github.com/BradenAnderson/Accident_Severity_Prediction/blob/main/Data/vehicle.csv.gz?raw=tr"
url_person = "https://github.com/BradenAnderson/Accident_Severity_Prediction/blob/main/Data/person.csv.gz?raw=tr"

accident = pd.read_csv(url_accident,compression='gzip')
vehicle = pd.read_csv(url_vehicle, compression='gzip', low_memory=False, encoding="ISO-8859-1")
person = pd.read_csv(url_person, compression='gzip', low_memory=False, encoding="ISO-8859-1")

In [3]:
# Filter accidents where driver is present and vehicle is involved
person = person.loc[(person.VEH_NO==1) & (person.PER_NO==1)]
vehicle = vehicle.loc[vehicle.VEH_NO==1]

In [4]:
# Left join person with vehicle and accident
# Duplicated CASENUM are dropped
df = person.merge(vehicle.drop_duplicates(subset=['CASENUM']), on='CASENUM', how='left')
df = df.merge(accident.drop_duplicates(subset=['CASENUM']),on='CASENUM',how='left')

In [5]:
# Comprehensive list of variables used in this analysis
# ORIGINAL features from lab 1 / EDA: regionname, urbanicityname, body_typname, makename, mod_yearname, vtrafwayname, vnum_lanname, vsurcondname, vtrafconname, 
                                # typ_intname, int_hwyname, weathername, wkdy_imname, reljct1_imname, lgtcon_imname, maxsev_imname, alchl_imname, age_im, sex_imname, trav_sp
# DERIVED features: hour_binned, speeding_status
# NEW features post-lab 1 / EDA: rest_usename, pcrash1_imname, weather_binned (binning of `weathername`), body_type_binned (binning of `body_typname`), int_binned (binning of `typ_intname`)
# DISCARDED features that were not useful: hour_imname (unnecessary with `hour_binned`), vspd_lim (unnecessary with `speeding_status`), makename (too many levels), wrk_zonename

df = df[['REGIONNAME','URBANICITYNAME','BODY_TYPNAME_x', 'MOD_YEARNAME_x','VTRAFWAYNAME','VNUM_LANNAME','VSURCONDNAME','VTRAFCONNAME','TYP_INTNAME','INT_HWYNAME','WEATHERNAME',
        'WKDY_IMNAME', 'RELJCT1_IMNAME','LGTCON_IMNAME','MAXSEV_IMNAME','ALCHL_IMNAME','AGE_IM','SEX_IMNAME','TRAV_SP','REST_USENAME','PCRASH1_IMNAME','HOUR_IMNAME','VSPD_LIM',
        'HOUR_IM']]

df = df.rename(columns=str.lower)
df.shape

(54473, 24)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54473 entries, 0 to 54472
Data columns (total 24 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   regionname      54473 non-null  object 
 1   urbanicityname  54473 non-null  object 
 2   body_typname_x  54473 non-null  object 
 3   mod_yearname_x  54473 non-null  object 
 4   vtrafwayname    54427 non-null  object 
 5   vnum_lanname    54427 non-null  object 
 6   vsurcondname    54427 non-null  object 
 7   vtrafconname    54427 non-null  object 
 8   typ_intname     54473 non-null  object 
 9   int_hwyname     54473 non-null  object 
 10  weathername     54473 non-null  object 
 11  wkdy_imname     54473 non-null  object 
 12  reljct1_imname  54473 non-null  object 
 13  lgtcon_imname   54473 non-null  object 
 14  maxsev_imname   54473 non-null  object 
 15  alchl_imname    54473 non-null  object 
 16  age_im          54473 non-null  int64  
 17  sex_imname      54473 non-null 

In [7]:
df.head()

,regionname,urbanicityname,body_typname_x,mod_yearname_x,vtrafwayname,vnum_lanname,vsurcondname,vtrafconname,typ_intname,int_hwyname,...,maxsev_imname,alchl_imname,age_im,sex_imname,trav_sp,rest_usename,pcrash1_imname,hour_imname,vspd_lim,hour_im
0,"West (MT, ID, WA, OR, CA, NV, NM, AZ, UT, CO, ...",Rural Area,"4-door sedan, hardtop",2018,"Two-Way, Not Divided",Five lanes,Snow,Traffic control signal(on colors) not known wh...,Four-Way Intersection,No,...,No Apparent Injury (O),No Alcohol Involved,61,Female,25.0,Shoulder and Lap Belt Used,Going Straight,8:00am-8:59am,98.0,8
1,"South (MD, DE, DC, WV, VA, KY, TN, NC, SC, GA,...",Urban Area,"4-door sedan, hardtop",2013,"Two-Way, Not Divided",Two lanes,Dry,No Controls,Not an Intersection,No,...,Suspected Minor Injury (B),No Alcohol Involved,23,Male,45.0,Shoulder and Lap Belt Used,Going Straight,1:00am-1:59am,25.0,1
2,"South (MD, DE, DC, WV, VA, KY, TN, NC, SC, GA,...",Urban Area,Other or Unknown automobile type,Unknown,"Two-Way, Divided, Unprotected Median",Four lanes,Dry,No Controls,T-Intersection,No,...,No Apparent Injury (O),No Alcohol Involved,27,Female,15.0,Not Reported,Going Straight,1:00pm-1:59pm,45.0,13
3,"West (MT, ID, WA, OR, CA, NV, NM, AZ, UT, CO, ...",Rural Area,"Compact Utility (Utility Vehicle Categories ""S...",2015,"Two-Way, Divided, Positive Median Barrier",Two lanes,Snow,No Controls,Not an Intersection,Yes,...,No Apparent Injury (O),No Alcohol Involved,20,Male,65.0,Shoulder and Lap Belt Used,Going Straight,2:00pm-2:59pm,80.0,14
4,"Northeast (PA, NJ, NY, NH, VT, RI, MA, ME, CT)",Rural Area,Station Wagon (excluding van and truck based),2004,Not Reported,Not Reported,Snow,Warning Sign,Not an Intersection,No,...,No Apparent Injury (O),No Alcohol Involved,23,Male,998.0,Shoulder and Lap Belt Used,Negotiating a Curve,5:00pm-5:59pm,50.0,17


In [8]:
# Check for NA values
df.isnull().sum()

regionname         0
urbanicityname     0
body_typname_x     0
mod_yearname_x     0
vtrafwayname      46
vnum_lanname      46
vsurcondname      46
vtrafconname      46
typ_intname        0
int_hwyname        0
weathername        0
wkdy_imname        0
reljct1_imname     0
lgtcon_imname      0
maxsev_imname      0
alchl_imname       0
age_im             0
sex_imname         0
trav_sp           46
rest_usename       0
pcrash1_imname    46
hour_imname        0
vspd_lim          46
hour_im            0
dtype: int64

In [9]:
df.shape

(54473, 24)

In [10]:
# Remove NA values since they are low representation of the entire dataset
df.dropna(inplace=True)

In [11]:
# Confirm NA values are removed
df.isnull().sum().sum()

0

In [12]:
df.shape

(54427, 24)

### Inspect each feature

#### Year of vehicle **(Continuous)**

In [13]:
# mod_yearname_x
# Remove unknown instances
# Filter to years 1980+
df.mod_yearname_x.value_counts()

2016       3809
2017       3722
2019       3489
2018       3356
Unknown    2935
           ... 
1972          1
1932          1
1953          1
1959          1
1969          1
Name: mod_yearname_x, Length: 67, dtype: int64

In [14]:
df.shape

(54427, 24)

In [13]:
# Remove unknown instances, convert to numeric dtype
non_int_values = pd.to_numeric(df.mod_yearname_x, errors='coerce').isna()
df = df[-non_int_values]
df['mod_yearname_x'] = pd.to_numeric(df['mod_yearname_x'])

In [15]:
# Filter to years 1980+
df = df.loc[df['mod_yearname_x'] >= 1980]

In [17]:
df.shape

(50930, 24)

#### Age of driver **(Continuous)**

In [29]:
# age_im
# Removed observations below 15 years old
# Should we bin 80+ or leave as is?
df = df.loc[df['age_im'] > 15]

In [30]:
df.shape

(50705, 24)

#### Traveling speed of vehicle **(Continuous)**

In [37]:
# trav_sp
# Speed Greater Than 151 mph indicated as 997
# Speed not reported indicated as 998
# Speed reported as unknown indicated as 999
# Discern number of rows lost if 997, 998, and 998 are removed
# 28975 !! Will impute instead
df.loc[df['trav_sp'] >= 997].shape[0]

28975

In [21]:
df.shape

(50705, 24)

#### Sex of driver

In [22]:
# sex_imname
df.sex_imname.value_counts()

Male      31519
Female    19186
Name: sex_imname, dtype: int64

#### Region where crash occurred

- Northeast (PA, NJ, NY, NH, VT, RI, MA, ME, CT)
- West (MT, ID, WA, OR, CA, NV, NM, AZ, UT, CO, WY, AK, HI)
- Midwest (OH, IN, IL, MI, WI, MN, ND, SD, NE, IA, MO, KS)
- South (MD, DE, DC, WV, VA, KY, TN, NC, SC, GA, FL, AL, MS, LA, AR, OK, TX)


In [40]:
# regionname
df["regionname"] = df.loc[:,"regionname"].apply(lambda string: string.split()[0])
df.regionname.value_counts()

/var/folders/0d/nkpt65qj6d16zs305flndkkc0000gn/T/ipykernel_74095/1409682747.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["regionname"] = df.loc[:,"regionname"].apply(lambda string: string.split()[0])


South        26481
Midwest       9643
West          8449
Northeast     6132
Name: regionname, dtype: int64

#### Geographical area of the crash

- Urban
- Rural

In [41]:
# urbanicityname
df["urbanicityname"] = df.loc[:,"urbanicityname"].apply(lambda string: string.split()[0])
df.urbanicityname.value_counts()

/var/folders/0d/nkpt65qj6d16zs305flndkkc0000gn/T/ipykernel_74095/809076839.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["urbanicityname"] = df.loc[:,"urbanicityname"].apply(lambda string: string.split()[0])


Urban    37472
Rural    13233
Name: urbanicityname, dtype: int64

#### Trafficway flow just prior to crash

In [25]:
# vtrafwayname
# Bin observations 'Not Reported', 'Reported as Unknown' into NR/UNK
df.vtrafwayname.value_counts()

Two-Way, Not Divided                                     21560
Not Reported                                              8451
Two-Way,  Divided, Positive  Median Barrier               8346
Two-Way, Divided, Unprotected Median                      6455
Two-Way, Not Divided With a Continuous Left-Turn Lane     1860
Non-Trafficway or Driveway Access                         1801
Entrance/Exit Ramp                                        1217
One-Way Trafficway                                         997
Reported as Unknown                                         18
Name: vtrafwayname, dtype: int64

In [26]:
df.shape

(50705, 24)

In [42]:
# Bin observations 'Not Reported', 'Reported as Unknown' into NR/UNK
df['vtrafwayname'] = df['vtrafwayname'].replace(['Not Reported', 'Reported as Unknown'], 'NR/UNK')

# Remove observations 'Not Reported', 'Reported as Unknown'
# remove = ['Not Reported', 'Reported as Unknown']
# df = df[df.vtrafwayname.isin(remove) == False]

/var/folders/0d/nkpt65qj6d16zs305flndkkc0000gn/T/ipykernel_74095/2192158929.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['vtrafwayname'] = df['vtrafwayname'].replace(['Not Reported', 'Reported as Unknown'], 'NR/UNK')


In [44]:
df.shape

(50705, 24)

In [43]:
df.vtrafwayname.value_counts()

Two-Way, Not Divided                                     21560
NR/UNK                                                    8469
Two-Way,  Divided, Positive  Median Barrier               8346
Two-Way, Divided, Unprotected Median                      6455
Two-Way, Not Divided With a Continuous Left-Turn Lane     1860
Non-Trafficway or Driveway Access                         1801
Entrance/Exit Ramp                                        1217
One-Way Trafficway                                         997
Name: vtrafwayname, dtype: int64

#### Number of travel lanes just prior to crash. 
- Median: lanes in opposite directions are additive. 
- No median: lanes in traveling direction counts

In [30]:
# vnum_lanname
# Bin 'Not Reported', 'Reported as Unknown' as NR/UNK
df.vnum_lanname.value_counts()

Two lanes                            17411
Not Reported                          6779
Three lanes                           5684
Four lanes                            4600
Five lanes                            3497
Non-Trafficway or Driveway Access     1801
Six lanes                             1169
One lane                               892
Seven or more lanes                    395
Reported as Unknown                      8
Name: vnum_lanname, dtype: int64

In [50]:
# Bin 'Not Reported', 'Reported as Unknown' as NR/UNK
df['vnum_lanname'] = df['vnum_lanname'].replace(['Not Reported', 'Reported as Unknown'], 'NR/UNK')

# Remove observations 'Not Reported', 'Reported as Unknown'
# remove = ['Not Reported', 'Reported as Unknown']
# df = df[df.vnum_lanname.isin(remove) == False]

/var/folders/0d/nkpt65qj6d16zs305flndkkc0000gn/T/ipykernel_74095/767845099.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['vnum_lanname'] = df['vnum_lanname'].replace(['Not Reported', 'Reported as Unknown'], 'NR/UNK')


In [51]:
df.shape

(50705, 24)

In [52]:
df.vnum_lanname.value_counts()

Two lanes                            18000
NR/UNK                               14356
Three lanes                           5775
Four lanes                            4714
Five lanes                            3535
Non-Trafficway or Driveway Access     1801
Six lanes                             1200
One lane                               917
Seven or more lanes                    407
Name: vnum_lanname, dtype: int64

#### Roadway surface condition just prior to crash

In [35]:
# vsurcondname
# Bin 'Not Reported', 'Reported as Unknown', 'Other' as NR/UNK/OTH
df.vsurcondname.value_counts()

Dry                                  26574
Wet                                   4787
Non-Trafficway or Driveway Access     1801
Not Reported                          1500
Snow                                   322
Ice/Frost                              227
Water (Standing or Moving)              80
Slush                                   65
Mud, Dirt or Gravel                     47
Reported as Unknown                     31
Sand                                    11
Other                                    2
Oil                                      2
Name: vsurcondname, dtype: int64

In [53]:
# Bin 'Not Reported', 'Reported as Unknown', 'Other' as NR/UNK/OTH
df['vsurcondname'] = df['vsurcondname'].replace(['Not Reported', 'Reported as Unknown', 'Other'], 'NR/UNK/OTH')

# Remove observations 'Not Reported', 'Reported as Unknown'
# remove = ['Not Reported', 'Reported as Unknown', 'Other']
# df = df[df.vsurcondname.isin(remove) == False]

/var/folders/0d/nkpt65qj6d16zs305flndkkc0000gn/T/ipykernel_74095/124740856.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['vsurcondname'] = df['vsurcondname'].replace(['Not Reported', 'Reported as Unknown', 'Other'], 'NR/UNK/OTH')


In [55]:
df.shape

(50705, 24)

In [54]:
df.vsurcondname.value_counts()

Dry                                  37880
Wet                                   6681
NR/UNK/OTH                            3231
Non-Trafficway or Driveway Access     1801
Snow                                   487
Ice/Frost                              325
Water (Standing or Moving)             102
Slush                                   92
Mud, Dirt or Gravel                     87
Sand                                    15
Oil                                      4
Name: vsurcondname, dtype: int64

#### Traffic controls in the vehicle’s environment just prior to crash

In [38]:
# vtrafconname
# Bin 'Not Reported', 'Other Regulatory Sign', 'Other', 'Reported as Unknown', 'Other Highway Traffic Signal', 'Unknown Highway Traffic Signal', 'Unknown Regulatory Sign'
df.vtrafconname.value_counts()

No Controls                                                                     17979
Traffic control signal(on colors) not known whether or not Pedestrian Signal     7064
Not Reported                                                                     5267
Stop Sign                                                                        2539
Yield Sign                                                                        417
Warning Sign                                                                      202
Traffic control signal (on colors) with Pedestrian Signal                         131
Other Regulatory Sign                                                              72
Flashing Traffic Control Signal                                                    69
Other                                                                              51
Railway Crossing Device                                                            36
Person                                                

In [56]:
# Bin 'Not Reported', 'Other Regulatory Sign', 'Other', 'Reported as Unknown', 'Other Highway Traffic Signal', 'Unknown Highway Traffic Signal', 'Unknown Regulatory Sign'
df['vtrafconname'] = df['vtrafconname'].replace(['Not Reported', 'Other Regulatory Sign', 'Other', 'Reported as Unknown', 'Other Highway Traffic Signal', 'Unknown Highway Traffic Signal', 'Unknown Regulatory Sign'], 'NR/UNK/OTH')

# Remove observations 'Not Reported', 'Other', 'Reported as Unknown', 'Unknown Highway Traffic Signal', 'Unknown Regulatory Sign'
# remove = ['Not Reported', 'Other', 'Reported as Unknown', 'Unknown Highway Traffic Signal', 'Unknown Regulatory Sign']
# df = df[df.vtrafconname.isin(remove) == False]

/var/folders/0d/nkpt65qj6d16zs305flndkkc0000gn/T/ipykernel_74095/3492617022.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['vtrafconname'] = df['vtrafconname'].replace(['Not Reported', 'Other Regulatory Sign', 'Other', 'Reported as Unknown', 'Other Highway Traffic Signal', 'Unknown Highway Traffic Signal', 'Unknown Regulatory Sign'], 'NR/UNK/OTH')


In [57]:
df.vtrafconname.value_counts()

No Controls                                                                     25691
Traffic control signal(on colors) not known whether or not Pedestrian Signal    10520
NR/UNK/OTH                                                                       7943
Stop Sign                                                                        4768
Yield Sign                                                                        580
Traffic control signal (on colors) with Pedestrian Signal                         520
Warning Sign                                                                      251
Traffic control signal (on colors) without Pedestrian Signal                      183
Flashing Traffic Control Signal                                                   128
Railway Crossing Device                                                            49
Person                                                                             47
School Zone Sign/Device                               

In [58]:
df.shape

(50705, 24)

#### Did crash occur at intersection

In [59]:
# int_hwyname
# Remove 'Unknown'
df.int_hwyname.value_counts()

No         46035
Yes         4663
Unknown        7
Name: int_hwyname, dtype: int64

In [60]:
# Drop observations with Unknown
df.drop(df[df['int_hwyname'] == 'Unknown'].index, inplace = True)
df.int_hwyname.value_counts()

/var/folders/0d/nkpt65qj6d16zs305flndkkc0000gn/T/ipykernel_74095/2412876005.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(df[df['int_hwyname'] == 'Unknown'].index, inplace = True)


No     46035
Yes     4663
Name: int_hwyname, dtype: int64

In [61]:
df.shape

(50698, 24)

#### Name of weekday where crash occurred

In [43]:
# wkdy_imname
df.wkdy_imname.value_counts()

Friday       4830
Wednesday    4362
Thursday     4353
Tuesday      4044
Monday       3929
Saturday     3873
Sunday       3188
Name: wkdy_imname, dtype: int64

#### Relation to junction (crash's location with respect to presence in an interchange area)

In [44]:
# reljct1_imname
df.reljct1_imname.value_counts()

No     26487
Yes     2092
Name: reljct1_imname, dtype: int64

#### Lighting condition during time of crash

In [45]:
# lgtcon_imname
df.lgtcon_imname.value_counts()

Daylight                   19381
Dark - Lighted              4670
Dark - Not Lighted          3170
Dusk                         737
Dawn                         424
Dark - Unknown Lighting      191
Other                          6
Name: lgtcon_imname, dtype: int64

#### Alcohol state of driver

In [46]:
# alchl_imname
df.alchl_imname.value_counts()

No Alcohol Involved    26533
Alcohol Involved        2046
Name: alchl_imname, dtype: int64

In [47]:
df.shape

(28579, 24)

### Newly derived features or careful binning of existing features

#### Speeding status of driver (Braden) (Derived feature)

In [48]:
def create_speeding_feature(row):
    
    # Speed greater than 151 mph, you're speeding
    if row["trav_sp"] == 997:
        return 'speeding'
    
    # If we know they were going 95 mph or faster, calling that speeding
    # regardless of what the speed limit is, or if we even know ths speed limit.
    elif row["trav_sp"] < 152 and row["trav_sp"] >= 95:
        return 'speeding'
    
    # If the speed limit is unknown, speeding is unknown
    elif row["vspd_lim"] == 98 or row["vspd_lim"] == 99:
        return 'unknown'
    
    # If the traveling speed is unknown, speeding is unknown
    elif row['trav_sp'] == 998 or row['trav_sp'] == 999:
        return 'unknown'
    
    # If traveling faster than the speed limit, speeding
    elif row['trav_sp'] > row['vspd_lim']:
        return 'speeding'
    else:
        return 'not speeding'
    
df["speeding_status"] = df.apply(lambda row: create_speeding_feature(row), axis='columns')

#### Time of accident, hours binned (Braden) (Derived feature)

- Morning (6am-noon)
- Afternoon (noon-6pm)
- Evening (6pm-midnight)
- Night (midnight-6am)']

In [49]:
# code here
def create_binned_hours_feature(numeric_hour, night_hours=[0,1,2,3,4,21,22,23], morning_hours=[5,6,7,8,9,10,11], 
                                afternoon_hours=[12,13,14,15], evening_hours=[16,17,18,19,20]):
    
    # NOTE: numeric_hour is a value from the HOUR_IM column

    # Default for night is 9pm-4:59am
    if numeric_hour in night_hours:
        return "Night"
    
    # Default for morning is 5am-11:59am
    elif numeric_hour in morning_hours:
        return "Morning"
    
    # Default for Afternoon is 12:00pm-3:59pm
    elif numeric_hour in afternoon_hours:
        return "Afternoon"
    
    # Default for evening is 4:00pm-8:59pm
    elif numeric_hour in evening_hours:
        return "Evening"
    
df["hour_binned"] = df["hour_im"].apply(lambda hour: create_binned_hours_feature(numeric_hour=hour))

# After hour_im is used to create hour_binned, we no longer need hour_im
df.drop(columns="hour_im", inplace=True)

#### Body type of vehicle (Tavin) (Careful binning)

In [50]:
# code here

#### Type of intersection (Careful binning)

In [51]:
df.typ_intname.value_counts()

Not an Intersection        15849
Four-Way Intersection       8072
T-Intersection              3218
Not Reported                1163
Five Point, or More           94
Y-Intersection                83
Roundabout                    68
Traffic Circle                19
L-Intersection                 8
Other Intersection Type        5
Name: typ_intname, dtype: int64

In [52]:
df.columns

Index(['regionname', 'urbanicityname', 'body_typname_x', 'mod_yearname_x',
       'vtrafwayname', 'vnum_lanname', 'vsurcondname', 'vtrafconname',
       'typ_intname', 'int_hwyname', 'weathername', 'wkdy_imname',
       'reljct1_imname', 'lgtcon_imname', 'maxsev_imname', 'alchl_imname',
       'age_im', 'sex_imname', 'trav_sp', 'rest_usename', 'pcrash1_imname',
       'hour_imname', 'vspd_lim', 'speeding_status', 'hour_binned'],
      dtype='object')

In [53]:
def intersection_category(row):
    if row == 'Not an Intersection':
        result = 'No'
    elif row == 'Reported as Unknown':
        result = 'Other'
    elif row == 'Not Reported':
        result = 'Other'
    else:
        result = 'Yes'
    return result

df['intersection_binned'] = df['typ_intname'].apply(intersection_category)
df.intersection_binned.value_counts()

No       15849
Yes      11567
Other     1163
Name: intersection_binned, dtype: int64

#### Weather during time of crash (Careful binning)

In [54]:
# weathername
df.weathername.value_counts()

Clear                       19944
Cloudy                       4251
Rain                         2910
Not Reported                  857
Snow                          392
Fog, Smog, Smoke              130
Severe Crosswinds              21
Sleet or Hail                  19
Reported as Unknown            19
Blowing Snow                   12
Freezing Rain or Drizzle       11
Other                          11
Blowing Sand, Soil, Dirt        2
Name: weathername, dtype: int64

In [56]:
def weather_cat(row):
    if row == 'Cloudy':
        return 'Not Clear'
    elif row == 'Fog, Smog, Smoke':
        return 'Not Clear' 
    elif row == 'Snow':
        return 'Wintery'
    elif row == 'Blowing Snow':
        return 'Wintery'
    elif row == 'Sleet or Hail':
        return 'Wintery'
    elif row == 'Freezing Rain or Drizzle':
        return 'Wintery'
    elif row == 'Severe Crosswinds':
        return 'Windy'
    elif row == 'Blowing Sand, Soil, Dirt':
        return 'Windy'
    elif row == 'Clear':
        return 'Clear'
    elif row == 'Rain':
        return 'Rain'
    else:
        return 'Other'
df['weather_binned'] = df['weathername'].apply(weather_cat)
df.weather_binned.value_counts()

Clear        19944
Not Clear     4381
Rain          2910
Other          887
Wintery        434
Windy           23
Name: weather_binned, dtype: int64

#### Restraint use by driver (Newly added feature)

In [57]:
# rest_use
df.rest_usename.value_counts()

rest_usename                 
Shoulder and Lap Belt Used       23489
None Used/Not Applicable          2628
Reported as Unknown               1549
Not Reported                       515
Shoulder Belt Only Used            237
Lap Belt Only Used                 123
Restraint Used - Type Unknown       34
Other                                4
dtype: int64

In [58]:
# rest_use: bin into none, minimal, full, other
def restraint_category(row):
    if row == 'Shoulder and Lap Belt Used':
        result = 'Full'
    elif row == 'None Used/Not Applicable':
        result = 'None'    
    elif row == 'Lap Belt Only Used':
        result = 'Minimal'
    elif row == 'Shoulder Belt Only Used':
        result = 'Minimal'
    elif row == 'Restraint Used - Type Unknown':
        result = 'Minimal'
    else:
        result = 'Other'
    return result

df['restraint_binned'] = df['rest_usename'].apply(restraint_category)
df.restraint_binned.value_counts()

restraint_binned
Full                23489
None                 2628
Other                2068
Minimal               394
dtype: int64

#### What driver was doing right before crash (Newly added feature)

In [59]:
# pcrash1_imname
df.pcrash1_imname.value_counts()

Going Straight                                                15830
Turning Left                                                   4068
Negotiating a Curve                                            2312
Changing Lanes                                                 1705
Turning Right                                                  1427
Stopped in Roadway                                              639
Backing Up (other than for Parking Position)                    611
Decelerating in Road                                            584
Passing or Overtaking Another Vehicle                           423
Starting in Road                                                346
Making a U-turn                                                 246
Merging                                                         127
Leaving a Parking Position                                       83
Accelerating in Road                                             61
Other(specify:)                                 

#### **Response variable:** Maximum injury severity of driver (Careful binning)

In [60]:
# maxsev_imname
df.maxsev_imname.value_counts()

No Apparent Injury (O)          12563
Possible Injury (C)              6479
Suspected Minor Injury (B)       5318
Suspected Serious Injury (A)     3319
Fatal Injury (K)                  809
Injured, Severity Unknown          91
Name: maxsev_imname, dtype: int64

In [61]:
# Remove Injured, Severity Unknown and Died Prior to Crash 
remove = ['Injured, Severity Unknown', 'Died Prior to Crash*']
df = df[df.maxsev_imname.isin(remove) == False]
df['maxsev_imname'] = df['maxsev_imname'].str[:-4]
df.maxsev_imname.value_counts()

No Apparent Injury          12563
Possible Injury              6479
Suspected Minor Injury       5318
Suspected Serious Injury     3319
Fatal Injury                  809
Name: maxsev_imname, dtype: int64

In [62]:
# Combine No Apparent Injury, Possible Injury, Suspected Minor Injury into one bin
# Combine Suspected Serious Injury and Fatal Injury into one bin
df['maxsev_binned'] = df['maxsev_imname'].replace(['Suspected Fatal Injury'], 'Fatal Injury')
df['maxsev_binned'] = df['maxsev_imname'].replace(['No Apparent Injury', 'Possible Injury', 'Suspected Minor Injury'], 'Not Fatal Injury')
df.maxsev_binned.value_counts()


Not Fatal Injury            24360
Suspected Serious Injury     3319
Fatal Injury                  809
Name: maxsev_binned, dtype: int64

### End of data cleaning. Review final dataset

In [63]:
# Remove features that are not useful for modeling or redundant post binning
df = df[df.columns.difference(['body_typname_x', 'typ_intname', 'weathername', 'rest_usename', 'hour_imname', 'vspd_lim'])]

In [64]:
# Check for duplicates
# Recall duplicated casenum was taken cared of during merge
# Here we check for duplicates of instances excluding `casenum`
df.duplicated().sum()

0

In [65]:
# Confirm there are zero NA values
df.isnull().sum().sum()

0

In [66]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28488 entries, 0 to 54471
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   age_im               28488 non-null  int64  
 1   alchl_imname         28488 non-null  object 
 2   hour_binned          28488 non-null  object 
 3   int_hwyname          28488 non-null  object 
 4   intersection_binned  28488 non-null  object 
 5   lgtcon_imname        28488 non-null  object 
 6   maxsev_binned        28488 non-null  object 
 7   maxsev_imname        28488 non-null  object 
 8   mod_yearname_x       28488 non-null  int64  
 9   pcrash1_imname       28488 non-null  object 
 10  regionname           28488 non-null  object 
 11  reljct1_imname       28488 non-null  object 
 12  restraint_binned     28488 non-null  object 
 13  sex_imname           28488 non-null  object 
 14  speeding_status      28488 non-null  object 
 15  trav_sp              28488 non-null 

In [67]:
display(df)

,age_im,alchl_imname,hour_binned,int_hwyname,intersection_binned,lgtcon_imname,maxsev_binned,maxsev_imname,mod_yearname_x,pcrash1_imname,...,sex_imname,speeding_status,trav_sp,urbanicityname,vnum_lanname,vsurcondname,vtrafconname,vtrafwayname,weather_binned,wkdy_imname
0,61,No Alcohol Involved,Morning,No,Yes,Daylight,Not Fatal Injury,No Apparent Injury,2018,Going Straight,...,Female,unknown,25.0,Rural,Five lanes,Snow,Traffic control signal(on colors) not known wh...,"Two-Way, Not Divided",Not Clear,Wednesday
1,23,No Alcohol Involved,Night,No,No,Dark - Not Lighted,Not Fatal Injury,Suspected Minor Injury,2013,Going Straight,...,Male,speeding,45.0,Urban,Two lanes,Dry,No Controls,"Two-Way, Not Divided",Clear,Wednesday
3,20,No Alcohol Involved,Afternoon,Yes,No,Daylight,Not Fatal Injury,No Apparent Injury,2015,Going Straight,...,Male,not speeding,65.0,Rural,Two lanes,Snow,No Controls,"Two-Way, Divided, Positive Median Barrier",Wintery,Wednesday
7,57,No Alcohol Involved,Morning,No,Yes,Daylight,Not Fatal Injury,No Apparent Injury,2016,Stopped in Roadway,...,Male,not speeding,0.0,Rural,Five lanes,Wet,Traffic control signal(on colors) not known wh...,"Two-Way, Not Divided",Rain,Thursday
10,60,No Alcohol Involved,Morning,Yes,No,Daylight,Suspected Serious Injury,Suspected Serious Injury,2018,Going Straight,...,Male,unknown,998.0,Rural,Three lanes,Wet,No Controls,"Two-Way, Divided, Positive Median Barrier",Not Clear,Thursday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54465,25,No Alcohol Involved,Morning,No,No,Dawn,Not Fatal Injury,Possible Injury,2018,Going Straight,...,Female,unknown,998.0,Rural,Two lanes,Dry,No Controls,"Two-Way, Not Divided",Clear,Thursday
54466,26,No Alcohol Involved,Morning,No,No,Dawn,Suspected Serious Injury,Suspected Serious Injury,2013,Going Straight,...,Male,unknown,998.0,Rural,Two lanes,Dry,No Controls,"Two-Way, Not Divided",Not Clear,Thursday
54467,61,No Alcohol Involved,Afternoon,No,No,Daylight,Not Fatal Injury,No Apparent Injury,2017,Going Straight,...,Female,unknown,998.0,Rural,Two lanes,Dry,No Controls,"Two-Way, Not Divided",Clear,Monday
54468,70,No Alcohol Involved,Evening,No,No,Daylight,Not Fatal Injury,Suspected Minor Injury,2019,Going Straight,...,Male,unknown,998.0,Rural,Two lanes,Dry,No Controls,"Two-Way, Not Divided",Not Clear,Wednesday


## Begin data preprocessing

In [68]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [69]:
# code to scale features
scaler = StandardScaler()

# Fit the scaler on the training set age column, and transform (scale) the training set age column
train_df.loc[:, ["age_im"]] = scaler.fit_transform(X=train_df.loc[:, ["age_im"]])

# Using means and standard deviations from training set, scale the test set
test_df.loc[:, ["age_im"]] = scaler.transform(X=test_df.loc[:, ["age_im"]])

In [70]:
# code for one hot encode
features_to_ohe = ["hour_binned", "intersection_binned", "lgtcon_imname", "pcrash1_imname", "regionname", 
                   "restraint_binned", "vtrafconname", "vtrafwayname", "weather_binned", "wkdy_imname", 
                   "vnum_lanname", "vsurcondname", "speeding_status"]

# Features that need to become binary
binary_features = ["urbanicityname", "alchl_imname", "reljct1_imname", "int_hwyname", "sex_imname"]

# List of all the features that are not being converted to binary, or being OneHotEncoder
# (Just a list of all features not in either of the two lists above).
non_encoded_features = [col for col in df.columns if col not in features_to_ohe + binary_features]

# Map each soon to be binary feature to its two initial values
binary_encoding_map = {feature:df[feature].unique().tolist() for feature in binary_features}

# Instantiate the OrdinalEncoder that will create the binary columns
ord_encoder = OrdinalEncoder(categories=[value for value in binary_encoding_map.values()])

# Map each feature that will be OneHotEncoded to a list of all possible categories that feature can take on. 
oridnal_encoding_map = {feature:df[feature].unique().tolist() for feature in features_to_ohe}

# Instantiate the OneHotEncoder, telling it what the possible categories are for each feature.
oh_encoder = OneHotEncoder(categories=[value for value in oridnal_encoding_map.values()]) 

encoders = ColumnTransformer(transformers=[("ordinal_encoder", ord_encoder, binary_features), 
                                           ("one_hot_encoder", oh_encoder, features_to_ohe)], 
                             remainder="passthrough",
                             sparse_threshold=0,
                             n_jobs=-1)

# Create a dataframe of all the encoded columns in the training set
train_encoded_features = pd.DataFrame(encoders.fit_transform(train_df.drop(columns=non_encoded_features)), 
                                      columns=encoders.get_feature_names_out())

# Remove the non-encoded versions, and add in the encoded-versions of all columns.
# Creates a version of the training set where all columns that need to be encoded are now encoded.
train_encoded_df = pd.concat(objs=[train_df.drop(columns=features_to_ohe+binary_features).reset_index(drop=True), 
                                   train_encoded_features.reset_index(drop=True)], 
                             axis='columns')

# Create a dataframe of all encoded (one hot and ordinal) columns for the test set
test_encoded_features = pd.DataFrame(encoders.transform(test_df.drop(columns=non_encoded_features)), 
                                     columns=encoders.get_feature_names_out())

# Remove the non-encoded versions, and add in the encoded-versions of all columns.
# Creates a version of the test set where all columns that need to be encoded are now encoded.
test_encoded_df = pd.concat(objs=[test_df.drop(columns=features_to_ohe+binary_features).reset_index(drop=True), 
                                   test_encoded_features.reset_index(drop=True)], 
                             axis='columns')

#### Imputing travel speed

In [71]:
# from lightgbm import LGBMRegressor
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Set the placeholder values (which need to be imputed) to NaN
train_encoded_df.loc[train_encoded_df["trav_sp"].isin([997, 998, 999]), "trav_sp"] = np.nan
test_encoded_df.loc[test_encoded_df["trav_sp"].isin([997, 998, 999]), "trav_sp"] = np.nan

# Set up version of training dataset to be used for imputing trav_sp
imputer_train_df = train_encoded_df.copy(deep=True)
imputer_train_df.drop(columns=["maxsev_binned"], inplace=True)
severity_ord_enc = OrdinalEncoder(categories=[["No Apparent Injury", "Possible Injury", "Suspected Minor Injury", "Suspected Serious Injury", "Fatal Injury"]])
imputer_train_df["maxsev_imname"] = severity_ord_enc.fit_transform(X=imputer_train_df["maxsev_imname"].to_numpy().reshape(-1,1))

# Setting the minimum and maximum value the imputer can impute for a traveling speed. 
min_trav_sp = 15
max_trav_sp = 151
impute_min_values = [-np.inf if col != "trav_sp" else min_trav_sp for col in imputer_train_df.columns]
impute_max_values = [np.inf if col != "trav_sp" else max_trav_sp for col in imputer_train_df.columns]

# Instantiate the iterative imputer
imputer = IterativeImputer(min_value=impute_min_values, 
                           max_value=impute_max_values,
                           random_state=42)

# Fit the iterative imputer on the training set, and impute the training set
imputer_train_result = imputer.fit_transform(X=imputer_train_df)

# Take the output of running iteratrive imputer on the train set, and convert it to a dataframe
# Save the imputed version of traveling speed to train_encoded_df
imputer_train_result_df = pd.DataFrame(imputer_train_result, 
                                       columns=imputer.get_feature_names_out())
train_encoded_df["trav_sp_imputed"] = imputer_train_result_df["trav_sp"]

# Set up the version of the test dataset that will run through the imputer
imputer_test_df = test_encoded_df.copy(deep=True)
imputer_test_df.drop(columns=["maxsev_binned"], inplace=True)
imputer_test_df["maxsev_imname"] = severity_ord_enc.transform(X=imputer_test_df["maxsev_imname"].to_numpy().reshape(-1,1))

# Run the test data through the imputer
imputer_test_result = imputer.transform(X=imputer_test_df)

# Convert the output of running the test data through iterative imputer to a dataframe
imputer_test_result_df = pd.DataFrame(imputer_test_result, 
                                      columns=imputer.get_feature_names_out())
test_encoded_df["trav_sp_imputed"] = imputer_test_result_df["trav_sp"]

In [72]:
# Dropping the Non-Imputed versions of travel speed, now that imputation has been performed
train_imputed_df = train_encoded_df.drop(columns="trav_sp")
test_imputed_df = test_encoded_df.drop(columns="trav_sp")

#### Scaling travel speed

In [73]:
speed_scaler = StandardScaler()

# Fit and transform the scaler on the training set
train_imputed_df["trav_sp_scaled"] = speed_scaler.fit_transform(train_imputed_df["trav_sp_imputed"].to_numpy().reshape(-1,1))

# Only transform on the test set (no data leakage)
test_imputed_df["trav_sp_scaled"] = speed_scaler.transform(test_imputed_df["trav_sp_imputed"].to_numpy().reshape(-1,1))

# Use these dataframes moving forward
# The final, fully preprocessed dataframes (all features encoded and imputed as needed)
# Using "pp" to stand for preprocessed
train_pp_df = train_imputed_df.drop(columns="trav_sp_imputed")
test_pp_df = test_imputed_df.drop(columns="trav_sp_imputed")

In [74]:
train_pp_df.head()

,age_im,maxsev_binned,maxsev_imname,mod_yearname_x,ordinal_encoder__urbanicityname,ordinal_encoder__alchl_imname,ordinal_encoder__reljct1_imname,ordinal_encoder__int_hwyname,ordinal_encoder__sex_imname,one_hot_encoder__hour_binned_Morning,...,one_hot_encoder__vsurcondname_Ice/Frost,one_hot_encoder__vsurcondname_Slush,one_hot_encoder__vsurcondname_Water (Standing or Moving),one_hot_encoder__vsurcondname_Sand,"one_hot_encoder__vsurcondname_Mud, Dirt or Gravel",one_hot_encoder__vsurcondname_Oil,one_hot_encoder__speeding_status_unknown,one_hot_encoder__speeding_status_speeding,one_hot_encoder__speeding_status_not speeding,trav_sp_scaled
0,1.785844,Not Fatal Injury,No Apparent Injury,2016,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-0.215690
1,0.878655,Not Fatal Injury,No Apparent Injury,2004,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.473884
2,-1.275918,Not Fatal Injury,No Apparent Injury,2011,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.006285
3,0.254963,Not Fatal Injury,Suspected Minor Injury,2008,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-0.353708
4,1.729144,Not Fatal Injury,Possible Injury,2019,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-0.493025


In [75]:
# table of features with representation (int, float, one-hot)
# will do after braden does OHE

In [76]:
# code for feature selection: random forest feature importance

In [77]:
# plot?

In [78]:
# code for feature selection: selectkbest

In [79]:
# plot?

In [80]:
# code for feature selection: mutual information

In [81]:
# plot?

# Data Preparation Part 2
Describe the final dataset that is used for classification/regression (include a description of any newly formed variables you created).
(5)

In [82]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28488 entries, 0 to 54471
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   age_im               28488 non-null  int64  
 1   alchl_imname         28488 non-null  object 
 2   hour_binned          28488 non-null  object 
 3   int_hwyname          28488 non-null  object 
 4   intersection_binned  28488 non-null  object 
 5   lgtcon_imname        28488 non-null  object 
 6   maxsev_binned        28488 non-null  object 
 7   maxsev_imname        28488 non-null  object 
 8   mod_yearname_x       28488 non-null  int64  
 9   pcrash1_imname       28488 non-null  object 
 10  regionname           28488 non-null  object 
 11  reljct1_imname       28488 non-null  object 
 12  restraint_binned     28488 non-null  object 
 13  sex_imname           28488 non-null  object 
 14  speeding_status      28488 non-null  object 
 15  trav_sp              28488 non-null 

In [83]:
display(df)

,age_im,alchl_imname,hour_binned,int_hwyname,intersection_binned,lgtcon_imname,maxsev_binned,maxsev_imname,mod_yearname_x,pcrash1_imname,...,sex_imname,speeding_status,trav_sp,urbanicityname,vnum_lanname,vsurcondname,vtrafconname,vtrafwayname,weather_binned,wkdy_imname
0,61,No Alcohol Involved,Morning,No,Yes,Daylight,Not Fatal Injury,No Apparent Injury,2018,Going Straight,...,Female,unknown,25.0,Rural,Five lanes,Snow,Traffic control signal(on colors) not known wh...,"Two-Way, Not Divided",Not Clear,Wednesday
1,23,No Alcohol Involved,Night,No,No,Dark - Not Lighted,Not Fatal Injury,Suspected Minor Injury,2013,Going Straight,...,Male,speeding,45.0,Urban,Two lanes,Dry,No Controls,"Two-Way, Not Divided",Clear,Wednesday
3,20,No Alcohol Involved,Afternoon,Yes,No,Daylight,Not Fatal Injury,No Apparent Injury,2015,Going Straight,...,Male,not speeding,65.0,Rural,Two lanes,Snow,No Controls,"Two-Way, Divided, Positive Median Barrier",Wintery,Wednesday
7,57,No Alcohol Involved,Morning,No,Yes,Daylight,Not Fatal Injury,No Apparent Injury,2016,Stopped in Roadway,...,Male,not speeding,0.0,Rural,Five lanes,Wet,Traffic control signal(on colors) not known wh...,"Two-Way, Not Divided",Rain,Thursday
10,60,No Alcohol Involved,Morning,Yes,No,Daylight,Suspected Serious Injury,Suspected Serious Injury,2018,Going Straight,...,Male,unknown,998.0,Rural,Three lanes,Wet,No Controls,"Two-Way, Divided, Positive Median Barrier",Not Clear,Thursday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54465,25,No Alcohol Involved,Morning,No,No,Dawn,Not Fatal Injury,Possible Injury,2018,Going Straight,...,Female,unknown,998.0,Rural,Two lanes,Dry,No Controls,"Two-Way, Not Divided",Clear,Thursday
54466,26,No Alcohol Involved,Morning,No,No,Dawn,Suspected Serious Injury,Suspected Serious Injury,2013,Going Straight,...,Male,unknown,998.0,Rural,Two lanes,Dry,No Controls,"Two-Way, Not Divided",Not Clear,Thursday
54467,61,No Alcohol Involved,Afternoon,No,No,Daylight,Not Fatal Injury,No Apparent Injury,2017,Going Straight,...,Female,unknown,998.0,Rural,Two lanes,Dry,No Controls,"Two-Way, Not Divided",Clear,Monday
54468,70,No Alcohol Involved,Evening,No,No,Daylight,Not Fatal Injury,Suspected Minor Injury,2019,Going Straight,...,Male,unknown,998.0,Rural,Two lanes,Dry,No Controls,"Two-Way, Not Divided",Not Clear,Wednesday


In [84]:
# table of feature description

# Model and Evaluation 1
- Choose and explain your evaluation metrics that you will use (i.e., accuracy, precision, recall, F-measure, or any metric we have discussed). 
- Why are the measure(s) appropriate for analyzing the results of your modeling? Give a detailed explanation backing up any assertions.
(10)

**Classification:** 
The evaluation metric to classify driver's injury severity is F1. This is an appropriate metric because the response, injury severity, is a categorical variable. Due to the class imbalance, metrics such as accuracy is not desirable because the model could have a high no-information-rate i.e., choose the most populous category and be correct most of the time.

**Regression:**
The evaluation metric to classify driver's age is MSE. This is an appropriate metric because the response, age, is a continuous variable.

# Model and Evaluation 2
- Choose the method you will use for dividing your data into training and testing splits (i.e., are you using Stratified 10-fold cross validation? Why?). 
- Explain why your chosen method is appropriate or use more than one method as appropriate. For example, if you are using time series data then you should be using continuous training and testing sets across time.
(10)

**Classification:** 
The method used to divide the data into train and test split is xxx because xxx.

**Regression:**
The method used to divide the data into train and test split is xxx because xxx. 

In [85]:
# code to split data here

# Model and Evaluation 3
- Create three different classification/regression models for each task (e.g., random forest, KNN, and SVM for task one and the same or different algorithms for task two). 
- Two modeling techniques must be new (but the third could be SVM or logistic regression). 
- Adjust parameters as appropriate to increase generalization performance using your chosen metric. 
- You must investigate different parameters of the algorithms!
(20)

## Classification

### Random Forest for Classification

### kNN for Classification

### SVM for Classification

## Regression

### Random Forest for Regression

### kNN for Regression

### SVM for Regression

# Model and Evaluation 4
- Analyze the results using your chosen method of evaluation. 
- Use visualizations of the results to bolster the analysis. 
- Explain any visuals and analyze why they are interesting to someone that might use this model.
(10)

We need one plot of the winning model from each task.

## Classification

## Regression

# Model and Evaluation 5
- Discuss the advantages of each model for each classification task, if any. 
- If there are not advantages, explain why. Is any model better than another? 
- Is the difference significant with 95% confidence? Use proper statistical comparison methods. 
- You must use statistical comparison techniques—be sure they are appropriate for your chosen method of validation as discussed in unit 7 of the course.
(10)

ROC plot of winning model from each task.

## Classification

## Regression

# Model and Evaluation 6 
- Which attributes from your analysis are most important? 
- Use proper methods discussed in class to evaluate the importance of different attributes. 
- Discuss the results and hypothesize about why certain attributes are more important than others for a given classification task.
(10)

## Classification

## Regression

# Deployment
- How useful is your model for interested parties (i.e., the companies or organizations that might want to use it for prediction)? 
- How would you measure the model's value if it was used by these parties? 
- How would your deploy your model for interested parties? 
- What other data should be collected? How often would the model need to be updated, etc.? 
(5)

1. How useful is your model for interested parties (i.e., the companies or organizations that might want to use it for prediction)? 

2. How would you measure the model's value if it was used by these parties? 


3. How would your deploy your model for interested parties? 


4. What other data should be collected? How often would the model need to be updated, etc.? 

# Exceptional Work
- You have free reign to provide additional analyses. 
- One idea: grid search parameters in a parallelized fashion and visualize the performances across attributes. 
- Which parameters are most significant for making a good model for each classification algorithm?
(10)

In [86]:
# grid search code